In [2]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
import torchmetrics.functional as metrics
import numpy as np

In [3]:
db_name = 'fashion-MNIST'

fashion_data = fetch_openml(name=db_name, parser='auto', as_frame=False)

In [10]:
print(f"data => {fashion_data['data'].dtype}, {fashion_data['data'].shape}")
print(f"target => {fashion_data['target'].dtype}, {fashion_data['target'].shape}")
print(f"feature_names => {fashion_data['feature_names']}\ntarget_names => {fashion_data['target_names']}")
print(f"categories => {fashion_data['categories']}")

data => int64, (70000, 784)
target => object, (70000,)
feature_names => ['pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5', 'pixel6', 'pixel7', 'pixel8', 'pixel9', 'pixel10', 'pixel11', 'pixel12', 'pixel13', 'pixel14', 'pixel15', 'pixel16', 'pixel17', 'pixel18', 'pixel19', 'pixel20', 'pixel21', 'pixel22', 'pixel23', 'pixel24', 'pixel25', 'pixel26', 'pixel27', 'pixel28', 'pixel29', 'pixel30', 'pixel31', 'pixel32', 'pixel33', 'pixel34', 'pixel35', 'pixel36', 'pixel37', 'pixel38', 'pixel39', 'pixel40', 'pixel41', 'pixel42', 'pixel43', 'pixel44', 'pixel45', 'pixel46', 'pixel47', 'pixel48', 'pixel49', 'pixel50', 'pixel51', 'pixel52', 'pixel53', 'pixel54', 'pixel55', 'pixel56', 'pixel57', 'pixel58', 'pixel59', 'pixel60', 'pixel61', 'pixel62', 'pixel63', 'pixel64', 'pixel65', 'pixel66', 'pixel67', 'pixel68', 'pixel69', 'pixel70', 'pixel71', 'pixel72', 'pixel73', 'pixel74', 'pixel75', 'pixel76', 'pixel77', 'pixel78', 'pixel79', 'pixel80', 'pixel81', 'pixel82', 'pixel83', 'pixel84', 'pixel85', '

In [12]:
feature = fashion_data['data']
target = fashion_data['target']

print(f'feature => {type(feature)}, {feature.shape}')
print(f'feature raw data =>\n{feature[:1]}\n')

print(f'target => {type(target)}, {target.shape}')
print(f'target raw data => {target[:1]}')

feature => <class 'numpy.ndarray'>, (70000, 784)
feature raw data =>
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   1   0   0  13  73   0   0   1   4   0   0   0
    0   1   1   0   0   0   0   0   0   0   0   0   0   0   0   0   3   0
   36 136 127  62  54   0   0   0   1   3   4   0   0   3   0   0   0   0
    0   0   0   0   0   0   0   0   6   0 102 204 176 134 144 123  23   0
    0   0   0  12  10   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0 155 236 207 178 107 156 161 109  64  23  77 130  72  15   0   0
    0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216 216 163
  127 121 122 146 141  88 172  66   0   0  

In [13]:
norm_feature = feature/255.

print(f'norm_feature => \n{norm_feature[:2]}')
print(f'norm_feature min => {norm_feature.min()}, max => {norm_feature.max()}')

norm_feature => 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
norm_feature min => 0.0, max => 1.0


In [15]:
norm_target = target.astype(int)
print(f'norm_target : {norm_target.dtype}, {np.unique(norm_target).size}개')

norm_target : int64, 10개


In [16]:
print(f'norm_target => {norm_target.shape}, {norm_target.ndim}D\n{norm_target[:5]}')

norm_target => (70000,), 1D
[9 0 0 3 0]


In [17]:
class DLDataset(Dataset):
    def __init__(self, x_data, y_data):
        super().__init__()
        self.feature = torch.FloatTensor(x_data)
        self.target=torch.LongTensor(y_data)
    
    def __len__(self):
        return self.target.shape[0]
    
    def __getitem__(self,index):
        return self.feature[index], self.target[index]


In [19]:
all_dataset = DLDataset(norm_feature, norm_target)

print(f'[all dataset] feature : {all_dataset.feature.shape}, target : {all_dataset.target.shape}')

[all dataset] feature : torch.Size([70000, 784]), target : torch.Size([70000])


In [21]:
seed_gen=torch.Generator().manual_seed(42)

TR_SIZE, VA_SIZE, TE_SIZE = 0.7, 0.1, 0.2

trainDS, validDS, testDS = random_split(all_dataset,
                                        [TR_SIZE, VA_SIZE, TE_SIZE],
                                        generator=seed_gen)

print(f'Train dataset length    : {len(trainDS)}개')
print(f'Validation dataset      : {len(validDS)}개')
print(f'Test dataset            : {len(testDS)}개')

Train dataset length    : 49000개
Validation dataset      : 7000개
Test dataset            : 14000개


In [22]:
BATCH_SIZE = 50
TRAIN_DL = DataLoader(trainDS, batch_size=BATCH_SIZE)
VALID_DL = DataLoader(validDS, batch_size=BATCH_SIZE)
TEST_DL = DataLoader(testDS, batch_size=BATCH_SIZE)

In [24]:
class MNISTModel(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.layer1=nn.Linear(in_dim,200)
        self.layer2=nn.Linear(200,100)
        self.layer3=nn.Linear(100,50)
        self.layer4=nn.Linear(50, out_dim)
        self.relu=nn.ReLU()
    
    def forward(self, x):
        y=self.layer1(x)
        y=self.relu(y)
        y=self.layer2(y)
        y=self.relu(y)
        y=self.layer3(y)
        y=self.relu(y)
        y=self.layer4(y)
        return y

In [25]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

IN_DIM, OUT_DIM = norm_feature.shape[1], np.unique(norm_target).size

MODEL = MNISTModel(IN_DIM,OUT_DIM).to(DEVICE)

OPTIMIZER = optim.SGD(MODEL.parameters())

LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)

EPOCHS=3

CLASSES = 10

In [26]:
def training(epoch, kind='valid'):
    MODEL.train()

    train_report=[[],[],[]]
    
    for idx, (feature, target) in enumerate(TRAIN_DL):
        feature, target = feature.to(DEVICE), target.to(DEVICE)

        pre_target = MODEL(feature)

        loss = LOSS_FN(pre_target, target)
        train_report[0].append(loss)

        acc = metrics.accuracy(pre_target, target, task='multiclass', num_classes=CLASSES)
        train_report[1].append(acc)

        f1 = metrics.f1_score(pre_target, target, task='multiclass', num_classes=CLASSES)
        train_report[2].append(f1)

        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()

    testing_type = 'Valid' if kind =='valid' else 'Test'

    loss_score = (sum(train_report[0])/BATCH_SIZE).item()
    acc_score = (sum(train_report[1])/BATCH_SIZE).item()
    f1_score = (sum(train_report[2])/BATCH_SIZE).item()
    print(f'\n{epoch} Train ] Loss => {loss_score:.3f} Acc => {acc_score:.3f} F1 => {f1_score:.3f}')
    
    return loss_score, acc_score, f1_score


In [27]:
def testing(epoch, kind='valid'):
    MODEL.eval()

    with torch.no_grad():
        test_report = [[],[],[]]
        dataloader = VALID_DL if kind=='valid' else TEST_DL
        for idx, (feature,target) in enumerate(TRAIN_DL):
            feature, target = feature.to(DEVICE), target.to(DEVICE)

            pre_target = MODEL(feature)

            loss = LOSS_FN(pre_target, target)
            test_report[0].append(loss)

            acc = metrics.accuracy(pre_target, target, task='multiclass', num_classes=CLASSES)
            test_report[1].append(acc)

            f1 = metrics.f1_score(pre_target, target, task='multiclass', num_classes=CLASSES)
            test_report[2].append(f1)

    testing_type = 'Valid' if kind == 'valid' else 'Test'

    loss_score = (sum(test_report[0])/BATCH_SIZE).item()
    acc_score = (sum(test_report[1])/BATCH_SIZE).item()
    f1_score = (sum(test_report[2])/BATCH_SIZE).item()
    print(f'{epoch} {test_report} ] Loss => {loss_score:.3f} Acc => {acc_score:.3f} F1 => {f1_score:.3f}\n')

    return loss_score, acc_score, f1_score

In [28]:
dir = '../data/model/'
filename = dir+'best_model1.pth'

import os
if not os.path.exists(dir):
    os.mkdir(dir)

In [31]:
train_ = {'loss':[], 'acc':[], 'f1':[]}
val_ = {'loss':[], 'acc':[], 'f1':[]}

pre_va_score = 100

for epo in range(EPOCHS):
    print(f'[Epoch {epo+1}/{EPOCHS}] ', end="")

    tr_score=training(epo)
    va_score=testing(epo)

    if pre_va_score > va_score[0]:
        torch.save(MODEL, filename)

    for idx, key in enumerate(train_):
        train_[key].append(tr_score[idx])
        val_[key].append(va_score[idx])
print("Done!")

[Epoch 1/3] 
0 Train ] Loss => 44.975 Acc => 2.396 F1 => 2.396
0 [[tensor(2.3045), tensor(2.3035), tensor(2.2726), tensor(2.2780), tensor(2.2919), tensor(2.2709), tensor(2.2897), tensor(2.2846), tensor(2.3230), tensor(2.2787), tensor(2.2802), tensor(2.2876), tensor(2.2726), tensor(2.2907), tensor(2.2888), tensor(2.3014), tensor(2.2823), tensor(2.2904), tensor(2.2694), tensor(2.2967), tensor(2.2809), tensor(2.3037), tensor(2.2824), tensor(2.2918), tensor(2.2807), tensor(2.2976), tensor(2.2854), tensor(2.3100), tensor(2.2864), tensor(2.2815), tensor(2.2674), tensor(2.2696), tensor(2.2875), tensor(2.2747), tensor(2.2914), tensor(2.2881), tensor(2.2922), tensor(2.2771), tensor(2.2875), tensor(2.2716), tensor(2.2932), tensor(2.2778), tensor(2.2968), tensor(2.2905), tensor(2.2883), tensor(2.2937), tensor(2.3089), tensor(2.2891), tensor(2.2947), tensor(2.2727), tensor(2.3085), tensor(2.2872), tensor(2.2901), tensor(2.2865), tensor(2.2894), tensor(2.3011), tensor(2.3003), tensor(2.2859), tenso

In [32]:
for idx, key in enumerate(train_):
    print(f'{key}', train_[key])

loss [44.9753532409668, 44.51213073730469, 43.61814498901367]
acc [2.3960046768188477, 4.219998359680176, 7.46560525894165]
f1 [2.3960046768188477, 4.219998359680176, 7.46560525894165]
